# hicetnunc.xyz open dataset and parsers

[Hic et nunc](http://hicetnunc.xyz) is a new eco-friendly [NFT](https://en.wikipedia.org/wiki/Non-fungible_token) marketplace, built on top of [Tezos](https://en.wikipedia.org/wiki/Tezos) blockchain.

It is especially popular in generative graphics and data viz community, so I've decided to share data and all scripts that I've made for https://hashquine.github.io/hicetnunc rating.

It is published under [CC BY](https://creativecommons.org/licenses/by/2.0/) license, so that it is even possible to sell NFTs that use that data (or modified scripts) as long as there is the following phrase somewhere in the token description: `based on @hashquine dataset`.

Since hic et nunc servers are already under an extreme load due to quick growth, I've reorganized code, so that all data is taken from Tezos blockchain and IPFS **without any calls** to the [hicetnunc.xyz](http://hicetnunc.xyz) website or API. 

## Data sources

* Blockchain transactions by [TzStats API](https://tzstats.com/docs/api#tezos-api) ([better-call.dev](https://better-call.dev) was not used in order not to interfere with hicetnunc backend).
* [IPFS](https://ru.wikipedia.org/wiki/IPFS) by [cloudflare-ipfs.com](https://cloudflare-ipfs.com/) and [ipfs.io](https://ipfs.io/) depending on mime type (same sources as in hicetnunc frontend).
* Wallet address owner metadata (name, Twitter etc.) from [api.tzkt.io](https://api.tzkt.io/#operation/Accounts_GetMetadata) (same source as in hicetnunc frontend).

## What data is available

* Money data: list of all purchases, prices and commissions.
* All NFTs raw files, their previews and thumbnails, although 3d files and interactive SVG/HTML files are not yet processed properly.
* Authors metadata [verified via tzkt.io](https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified) like Twitter account address.
* Token transfers: list of changes of tokens owners including burns and direct transfers.
* All metadata available for tokens.
* Swaps and mints.

Data not available:
* Everything connected with [hDAO tokens](https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#what-are-those-little-circles-on-each-post-hdao-what-is-that) and [hDAO feed](https://www.hicetnunc.xyz/hdao). Although all related transactions are already being collected, they are not analysed yet.
* Twitter statistics like the number of followers.
* Direct money transfers between users, when NFT tokens are not transferred in the same transaction.

## Dataset schema

The goal was to simplify data analysis and visualization with a wide range of existing tools, so there are lots of redundant fields, which contain precalculated aggregations and different representations of the same data.

All files have two equivalent versions: JSON and CSV.
* JSON files are dictionary of dictionaries with rows of CSV files are indexed by the `*_id` field.
* CSV files have commas as delimiters.
* Fields values are ether numbers or strings, empty values represented by `-1` or `""`.
* All identifiers are strings.

Any field, which references some event in the blockchain (for example, mint time) have 4 representations:
* `mint_iso_date` &mdash; string with UTC date and time: `"2021-03-01T15:00:00Z"`,
* `mint_stamp` &mdash; integer Unix timestamp in seconds: `1614610800`,
* `mint_hash` &mdash; string with transaction hash, where event occurred: `"oom5Ju6X9nYpBCi..."`,
* `mint_row_id` &mdash; integer with global unique operation id (internal to TzStats) with that event: `42181049`

Any field, which references a set of values (like the set of prices of sold works), have following aggregations:
* `sold_count` &mdash; values count (excl. zeros),
* `sold_zero_count` &mdash; number of zeros,
* `sold_price_min` &mdash; minimum value (excl. zeros),
* `sold_price_max` &mdash; maximum value,
* `sold_price_sum` &mdash; sum of values,
* `sold_price_avg` &mdash; average value (sum divided by count excl. zeros).


## Tokens db schema

In [1]:
tokens_db_schema = {
'main': [{
    'field': 'token_id',
    'type': 'string',
    'example': '152',
    'description': (
        'Token identifier: numeric strings with OBJKT ids from '
        '<code>https://www.hicetnunc.xyz/objkt/{token_id}</code>.'
    ),
}, {
    'field': 'creator',
    'type': 'address',
    'description': 'Address of the token minter.',
}, {
    'field': 'mint_count',
    'type': 'integer',
    'example': 100,
    'description': (
        'How many token instances were minted. '
        'This number does not change: it is impossible to mint additional token instances in hicetnunc for now.'
    )
}, {
    'field': 'mint',
    'type': 'event',
    'description': 'Mint event.',
}, {
    'field': 'artifact_ipfs',
    'type': 'ipfs',
    'description': 'IPFS address of NFT content.',
}, {
    'field': 'artifact_mime',
    'type': 'string',
    'example': 'image/gif',
    'description': 'Mime type of <code>artifact_ipfs</code>.'
}, {
    'field': 'artifact_file_size',
    'type': 'integer',
    'example': 2043418,
    'description': 'File size of <code>artifact_ipfs</code> in bytes.',
}, {
    'field': 'info_title',
    'type': 'string',
    'example': 'Dali tower',
    'description': 'Title of the NFT specified by creator.',
}, {
    'field': 'info_description',
    'type': 'string',
    'example': 'generated by ...',
    'description': 'Description of the NFT specified by creator.',
}, {
    'field': 'info_tags',
    'type': 'string',
    'example': 'tag1    tag2',
    'description': 'List of the NFT tags specified by creator delimited with tab character.',
}],
'statistics': [{
    'field': 'author_sold_prices',
    'type': 'float_set',
    'description': (
        'Prices of sold works (in swaps) by author in XTZ. <br><br>'
        'Note, that theoretically single NFT instance can be sold by author multiple times '
        'in case the buyer returns it back.'
    )
}, {
    'field': 'secondary_sold_prices',
    'type': 'float_set',
    'description': 'Prices of sold works (in swaps) by other users in XTZ.',
}, {
    'field': 'available_prices',
    'type': 'float_set',
    'description': 'Prices of available works (in swaps) both from author and on secondary market in XTZ.',
}, {
    'field': 'burn_count',
    'type': 'integer',
    'example': 15,
    'description': 'How many token instances were burned by creator or other users.',
}, {
    'field': 'author_owns_count',
    'type': 'integer',
    'example': 5,
    'description': 'How many token instances are owned by author (not including <code>available_count</code>).',
}, {
    'field': 'other_own_count',
    'type': 'integer',
    'example': 95,
    'description': 'How many token instances are owned by other users (not by burn, swap or author).',
}, {
    'field': 'author_sent_count',
    'type': 'integer',
    'example': 5,
    'description': (
        'How much instances were directly transferred by author to users without swaps.'
    ),
}],
'auxiliary': [{
    'field': 'info_ipfs',
    'type': 'ipfs',
    'description': 'IPFS identifier of the NFT JSON metadata',
}, {
    'field': 'display_uri_ipfs',
    'type': 'ipfs',
    'description': (
        'Token preview image identifier on IPFS or empty string. '
        'Note, that it is rare and only used for specific file types like HTML'
    ),
}, {
    'field': 'royalties',
    'type': 'float',
    'example': 10.0,
    'description': (
        '"Royalties" parameter passed during token mint in percent. '
    ),
}, {
    'field': 'info_creator',
    'type': 'address',
    'description': (
        'Address of the token creator as specified in token meta JSON file. '
        'In 99.99% the same as <code>creator</code> field, but sometimes empty probably due to some bug.'
    ),
}, {
    'field': 'mint_ah_row_id',
    'type': 'integer',
    'description': (
        'Row id of mint operation by "Art house" contract (not by NFT contract).'
    ),
}]}

## Addrs db schema

In [2]:
address_db_schema = {
'main': [{
    'field': 'address',
    'type': 'address',
    'description': (
        'Address of the user in Tezos blockchain. Can be contract address as well.'
    ),
}, {
    'field': 'first_action',
    'type': 'event',
    'description': 'When first transaction (related to hicetnunc) with this address occurred.'
}, {
    'field': 'tzkt_info_name',
    'type': 'string',
    'example': 'dacosta works',
    'description': (
        'User alias field according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_twitter',
    'type': 'string',
    'example': 'dacosta_works',
    'description': (
        'User Twitter account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_email',
    'type': 'string',
    'example': 'sartist@gmail.com',
    'description': (
        'User e-mail according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_instagram',
    'type': 'string',
    'example': 'dacosta_works',
    'description': (
        'User Instagram account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_site',
    'type': 'string',
    'example': 'https://jsh.sh',
    'description': (
        'User website according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_description',
    'type': 'string',
    'example': 'NFT artist ...',
    'description': (
        'User description according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_logo',
    'type': 'string',
    'example': 'huson.png',
    'description': (
        'User logo according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>. Add prefix <code>https://services.tzkt.io/v1/avatars2/{logo}</code> '
        'to get the full url'
    )
}, {
    'field': 'tzkt_info_github',
    'type': 'string',
    'example': 'josim',
    'description': (
        'User GitHub account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_telegram',
    'type': 'string',
    'example': 'thebaskia',
    'description': (
        'User Telegram account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_facebook',
    'type': 'string',
    'example': 'barbeaucodeart',
    'description': (
        'User Facebook account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}, {
    'field': 'tzkt_info_reddit',
    'type': 'string',
    'example': 'user/mathmakesart',
    'description': (
        'User Reddit account according to '
        '<a href="https://github.com/hicetnunc2000/hicetnunc/blob/main/FAQ.md#how-to-get-verified">'
        'tzkt.io metadata</a>.'
    )
}],
'statistics': [{
    'field': 'bought_prices',
    'type': 'float_set',
    'description': (
        'Prices of bought works (in swaps) by this address in XTZ.'
    )
}, {
    'field': 'author_sold_prices',
    'type': 'float_set',
    'description': (
        'Prices of sold works (in swaps) by author in XTZ.'
    )
}, {
    'field': 'secondary_sold_prices',
    'type': 'float_set',
    'description': (
        'Prices of sold works on secondary market (i.e. swaps not by author) in XTZ.'
    )
}, {
    'field': 'available_prices',
    'type': 'float_set',
    'description': (
        'Prices of available works (in swaps) both by author and other users in XTZ.'
    )
}, {
    'field': 'in_tr_count',
    'type': 'integer',
    'example': 25,
    'description': (
        'Number of incoming transactions'
    )
}, {
    'field': 'out_tr_count',
    'type': 'integer',
    'example': 33,
    'description': (
        'Number of outcoming transactions'
    )
}, {
    'field': 'money_received',
    'type': 'float',
    'example': 2.78999,
    'description': (
        'Total volume of XTZ received by address in transactions related to hicetnunc'
    )
}, {
    'field': 'money_sent',
    'type': 'float',
    'example': 10,
    'description': (
        'Total volume of XTZ sent by address in transactions related to hicetnunc'
    )
}],
'auxiliary': [{
    'field': 'first_tr_has_reveal',
    'type': 'boolean',
    'description': 'Whether first interaction with hicetnunc was the first transaction in Tezos for user'
}]}

## Swaps db schema

In [3]:
swaps_db_schema = {
'main': [{
    'field': 'swap_id',
    'type': 'string',
    'example': '503',
    'description': (
        'Numeric string with swap identifier.'
    ),
}, {
    'field': 'token_id',
    'type': 'string',
    'example': '207',
    'description': (
        'Numeric string with token identifier.'
    ),
}, {
    'field': 'price',
    'type': 'float',
    'example': 1.5,
    'description': 'Price per item in XTZ.'
}, {
    'field': 'total_count',
    'type': 'integer',
    'example': 100,
    'description': 'Total number of instances in swap.'
}, {
    'field': 'created',
    'type': 'event',
    'description': 'When swap was created.'
}, {
    'field': 'closed',
    'type': 'event',
    'description': 'When swap was closed (cancelled). If closed, swap cannot be reopened again.'
}, {
    'field': 'is_secondary',
    'type': 'boolean',
    'example': 1,
    'description': 'If 1 - swap was created by other user, so it is the secondary market.'
}, {
    'field': 'created_by',
    'type': 'address',
    'description': 'Address of the user, which created swap.'
}],
'statistics': [{
    'field': 'sold_count',
    'type': 'integer',
    'example': 99,
    'description': 'Number of sold (including with zero price) instances.'
}, {
    'field': 'available_count',
    'type': 'integer',
    'example': 1,
    'description': 'Number of available instances if the swap is not closed, 0 otherwise.'
}, {
    'field': 'returned_count',
    'type': 'integer',
    'example': 0,
    'description': 'Number of unsold instances returned to the swap creator if the swap was closed, 0 otherwise.'
}, {
    'field': 'sold_price_sum',
    'type': 'float',
    'example': 148.5,
    'description': 'Total price of sold instances in XTZ.',
}],
# 'auxiliary': [{}]
}

## Sells db schema

In [4]:
sells_db_schema = {
'main': [{
    'field': 'tr',
    'type': 'event',
    'description': (
        'When transaction occurred.'
    ),
}, {
    'field': 'token_id',
    'type': 'string',
    'example': '205',
    'description': (
        'Numeric string with token identifier.'
    ),
}, {
    'field': 'count',
    'type': 'integer',
    'example': 10,
    'description': 'Number of token instances transferred.'
}, {
    'field': 'seller',
    'type': 'address',
    'description': 'Seller address i.e. user who created the swap'
}, {
    'field': 'buyer',
    'type': 'address',
    'example': 'tz1MwvWa...',
    'description': 'Buyer address',
}, {
    'field': 'price',
    'type': 'float',
    'example': 2.5,
    'description': 'Price per item in XTZ (how much buyer payed per item)'
}, {
    'field': 'swap_id',
    'type': 'string',
    'example': '503',
    'description': (
        'Numeric string with swap identifier.'
    ),
}, {
    'field': 'is_secondary',
    'type': 'boolean',
    'example': 1,
    'description': 'If 1 - swap was created by other user, so it is the secondary market.'
}],
'statistics': [{
    'field': 'total_royalties',
    'type': 'float',
    'example': 12.75,
    'description': (
        'Author income from royalties (already multiplied by <code>count</code>) for secondary sells and 0 otherwise'
    )
}, {
    'field': 'total_comission',
    'type': 'float',
    'example': 1.3,
    'description': 'Comission per item in XTZ. Usually 2.5% of price.',
}, {
    'field': 'total_seller_income',
    'type': 'float',
    'example': 1.3,
    'description': (
        'Seller income (already multiplied by <code>count</code>).'
    )
}]}

## Transfers db schema

In [5]:
transfers_db_schema = {
'main': [{
    'field': 'tr',
    'type': 'event',
    'description': (
        'When transaction occurred.'
    ),
}, {
    'field': 'category',
    'type': 'string',
    'example': ('</code><br><code>'.join([
        'author-&gt;swap',
        'swap-&gt;author',
        'swap-&gt;user',
        'user-&gt;user',
        'author-&gt;trash',
        'user-&gt;trash',
        'user-&gt;swap',
    ])),
    'description': (
        'String, which characterizes transfer in following terms: '
        '<code>author</code>, <code>swap</code>, <code>user</code>, <code>trash</code>. <br>'
        'This division is a convention used in this repository only.'
    )
}, {
    'field': 'token_id',
    'type': 'string',
    'example': '207',
    'description': (
        'Numeric string with token identifier.'
    ),
}, {
    'field': 'price',
    'type': 'float',
    'example': 1.5,
    'description': 'Price per item in XTZ or 0 for direct transfers.'
}, {
    'field': 'count',
    'type': 'integer',
    'example': 5,
    'description': 'Number of token instances transferred.'
}, {
    'field': 'swap_id',
    'type': 'string',
    'example': '523',
    'description': (
        'Numeric string with swap identifier or empty string if it is a direct transfer.'
    ),
}, {
    'field': 'sender',
    'type': 'address',
    'description': 'Sender address or empty string in case of mint'
}, {
    'field': 'receiver',
    'type': 'address',
    'example': 'tz1MwvWa...',
    'description': 'Receiver address',
}]
}

## Make readme.md

In [6]:
import textwrap

def db_fields_schema_to_md(schema):
    rows = []
    for section_id, section_rows in schema.items():
        section_title = {
            'main': None,
            'statistics': 'Statistics fields',
            'auxiliary': 'Auxiliary fields',
        }[section_id]
        if section_title:
            rows.append(f'    <td colspan="4" align="center"><b>{section_title}</b></td>\n')

        for row in section_rows:
            field_id = row['field']
            field_type = row['type']
            field_example = row.get('example')
            field_descr = row['description']
            
            if field_type == 'ipfs':
                field_example = field_example or 'Qma11k...'
                field_type = 'string'

            elif field_type == 'boolean':
                field_example = '1</code> <code>0'

            elif field_type == 'address':
                field_example = field_example or 'tz1UBZUk...'
                field_type = 'string'

            elif field_type == 'event':
                field_id = '</code><br><code>'.join([
                    f'{field_id}{postfix}'
                    for postfix in [
                        '_iso_date',
                        '_stamp',
                        '_hash',
                        '_row_id',
                    ]
                ])
                field_example = field_example or '</code><br><code>'.join([
                    '"2021-03-01T15:00:03Z"',
                    '1614610803',
                    '"oom5Ju6X9nYpBCi..."',
                    '42181049',
                ])
                field_type = '</code><br><code>'.join([
                    'string',
                    'integer',
                    'string',
                    'integer',
                ])

            elif field_type == 'float_set':
                field_id_stem = '_'.join(field_id.split('_')[:-1])
                field_id = '</code><br><code>'.join([
                    f'{field_id_stem}_count',
                    f'{field_id_stem}_zero_count',
                    f'{field_id}_min',
                    f'{field_id}_max',
                    f'{field_id}_sum',
                    f'{field_id}_avg',
                ])
                field_example = field_example or '</code><br><code>'.join([
                    '100',
                    '10',
                    '1.25',
                    '2.5',
                    '33.7',
                    '1.325',
                ])
                field_type = '</code><br><code>'.join([
                    'integer',
                    'integer',
                    'float',
                    'float',
                    'float',
                    'float',
                ])

            if field_type == 'string':
                if not field_example or '<code>' not in field_example:
                    field_example = f'"{field_example}"'

            row_md = ''
            row_md += f'<td><code>{field_id}</code></td>\n'
            row_md += f'<td><code>{field_type}</code></td>\n'
            row_md += f'<td><code>{field_example}</code></td>\n'
            row_md += f'<td>{field_descr}</td>\n'
            rows.append(textwrap.indent(row_md, '    '))

    rows = '</tr><tr>\n'.join(rows)
    return textwrap.dedent('''
        <table><tr>
            <th>field</th>
            <th>type</th>
            <th>example</th>
            <th>description</th>
        </tr><tr>
    ''').strip() + rows + '</tr></table>'


import json
from pathlib import Path

nb_json = json.loads(Path('./210329_dataset_schema.ipynb').read_text('utf-8'))

assert nb_json['cells'][0]['cell_type'] == 'markdown'
readme_intro = ''.join(nb_json['cells'][0]['source'])

Path('../README.md').write_text(readme_intro + f'''

### [tokens.json](./dataset/tokens.json) and [tokens.csv](./dataset/tokens.csv) &mdash; of all NFTs tokens

There is a confusing fact, that in hicetnunc each NFT can have multiple identical instances, which are fungible.
In this document term "token" refers to the set of all that instances.

There are following invariants:
<pre>mint_count = author_owns_count + available_count + other_own_count + burn_count
author_sent_count <= other_own_count</pre>

{db_fields_schema_to_md(tokens_db_schema)}

### [addrs.json](./dataset/addrs.json) and [addrs.csv](./dataset/addrs.csv) &mdash; of all hicetnunc users

All users, who ever created or owned NFT token.

{db_fields_schema_to_md(address_db_schema)}

### [sells.json](./dataset/sells.json) and [sells.csv](./dataset/sells.csv) &mdash; of all purchases via swaps

There is the following invariant:
<pre>price * count = total_royalties + total_comission + total_seller_income</pre>

{db_fields_schema_to_md(sells_db_schema)}

### [transfers.json](./dataset/transfers.json) and [transfers.csv](./dataset/transfers.csv) &mdash; all token transfers

{db_fields_schema_to_md(transfers_db_schema)}

### [swaps.json](./dataset/swaps.json) and [swaps.csv](./dataset/swaps.csv) &mdash; all swaps ever created

{db_fields_schema_to_md(swaps_db_schema)}

''', 'utf-8')

26088

In [11]:
datasets_fields = {
    'tokens': tokens_db_schema,
    'addrs': address_db_schema,
    'sells': sells_db_schema,
    'transfers': transfers_db_schema,
    'swaps': swaps_db_schema,
}

import json

schemas_cache_dir = Path('../cache/schemas')
(schemas_cache_dir / 'datasets_fields.json').write_text(
    json.dumps(datasets_fields, ensure_ascii=False, indent=4),
    'utf-8',
)

18458